### Why we used Gaussian Distribution

#### a Gaussian Distrubution:

<img src="../images/gaussianPdf.png" width=50%, height=50%>

#### a non Gaussian Distribution:

<img src="../images/nonGaussianPdf.png" width=50%, height=50%>

### How do you get the vector OI equations?

- Assume that the final analysis state can be written: $$ \vec{x}_a = \vec{x}_b + K\left(\vec{z}-H\vec{x}_b\right) $$ where $K$ is a yet-to-be-determined matrix.

- The variance can be written $$ \Gamma_a = \left< \left(x_a-\mu_a\right)^T\left(x_a-\mu_a\right)\right> $$

- The "total variance", similar to when we looked at with EOFs is: $$ Tr(\Gamma_a) = \left(\Gamma_a\right)_{ij}\delta_{ij} $$

- Now all we have to do is minimize the $Tr(\Gamma_a)$ with respect to $K$ by implicitly finding $K$ such that: $$ \frac{\partial}{\partial K} Tr(\Gamma_a) = 0 $$

- Actually solving for $K$ involves a number of matrix identities, probably the most important being the Sherman-Morrison-Woodbury formula in the form:

$$ \left( \Gamma_b^{-1} + H^T \Gamma_r ^{-1} H \right)^{-1} H^T \Gamma_r = \Gamma_b H^T \left(H \Gamma_b H^T + \Gamma_r \right) ^{-1} $$

### Optimal Interpolation as a "Least Squares" Problem

- Neglecting some normalization factors, the final a posterori probability that we are finding is $$P\left(x_a|z,x_b\right) \propto \exp(-J)$$

$$ J = \frac{1}{2}\left(\vec{x}_a - \vec{x}_b\right)^T \Gamma_b^{-1} \left(\vec{x}_a - \vec{x}_b\right) + \frac{1}{2}\left(\vec{z} - H \vec{x}_a\right)^T \Gamma_r^{-1} \left(\vec{z} - H \vec{x}_a\right)  $$


- This is similar to the case of "least squares", but with 2 terms now, 1 representing the error between the analysis and observations, and 1 representing the perturbation to the analysis state itself. If we were to choose $x_a$ to minimize $J$ we would have a type of analysis known as 3D var (section 5.4)

- Note that we did not minimize this cost function directly in order to get the vector OI equations, instead we minimized the variance directly. Technically these are two different estimators:
  1. Vector OI - minmizing the variance of the solution  
  2. 3DVAR - finding the maximum probability (mode)

- For Gaussian distributions these are exactly the same, so theoretically we can use either.
  - Practically this doesn't always work so well, more on that later. 


### More on the Observation operator

- The most famous example of this is that we have a satellite that measures radiance in the atmosphere. However our weather forecast model will usually only output the temperatures, so we would need something like:

$$ \vec{x}_{rad} = H (\vec{x}_{temp}) $$

- However the measurements won't the same as the modeled radiances, they will include some deviation:

$$ \vec{z}_{rad} = \vec{x}_{rad} + R\vec{u}$$

- The $R\vec{u}$ term is the "measurement noise". Its our representation of the fact that the model 

- In this case H would be a non-linear model, but we can linearize it so that we get a linear measurement equation:

$$ \vec{z}_{rad} = H|_{x_b} \vec{x}_{temp} + R\vec{u}$$

- An important aspect of this is that $\vec{z}_{rad}$ and $\vec{x}_{temp}$ do not have to have the same sizes! 


### What is the shape of all of this stuff?

The vector OI equations are:

$$ \begin{align*} K = & \Gamma_b H^T \left(H\Gamma_bH^T+\Gamma_r\right)^{-1} \\
\vec{\mu}_a = & \vec{\mu}_b + K\left(\vec{z}-H\vec{\mu}_b\right) \\
\Gamma_a= &\left(I-KH\right)\Gamma_b \end{align*} $$

- Assume that $\mu$ has shape nx1, and that $z$ has shape px1.
- What are the shapes of $H$, $\Gamma_b$, $\Gamma_r$, and $K$?

### What is the role of the covariances in OI?

- This is easiest to see with the scalar OI analysis equation:
$$\begin{align*}x_a = &  \frac{\sigma_B^2}{\sigma_R^2+\sigma_B^2} z + \left(1-\frac{\sigma_B^2}{\sigma_R^2+\sigma_B^2}\right)x_b \\
= & \frac{\sigma_B^2}{\sigma_R^2+\sigma_B^2} z + \frac{\sigma_R^2}{\sigma_R^2+\sigma_B^2}x_b \end{align*}$$

- $x_a$ is a weighted average of $z$ and $x_b$, and the weights are determined by the relative certainty in $z$ and $x_b$
  - e.g. if $\sigma_B \gg \sigma_R$ then $x_a \approx z$.    

- Imagine that we set $\sigma_B' = q \sigma_B$ and $\sigma_R' =  q \sigma_R$. This will have no impact on $x_a$! 

- However it will change the final analysis covariance: ${\sigma_a '}^2 = (1-K){\sigma_b '}^2 = q \sigma_a $

- So to understand the errors its best to look at $\sigma_a$ to understand the uncertainties in our estimate. 

### What is the code example in the notes doing? 

- This example will make much more sense if you look at the labs solution 2.1 as well as the Kalman Filtering example. 
  - Actually how I made this was I did the Kalman filtering example first, and then I downgraded it into an OI code, hence the naming convection. 

- Remember that the equation that we are simulating looks like:
$$ \begin{align*} x_{k} = & x_{k-1} -\frac{dt}{\tau} x_{k-1} + B \sqrt{dt} w_k \\ 
  z_k = & x_k + R u_k \end{align*}$$

- Numerically, we can solve this as $$x_k = \exp\left(-\frac{dt}{\tau}\right) x_{k-1} + B \sqrt{dt} w_k$$
  In the code, Am = $\exp\left(-\frac{dt}{\tau}\right)$

- The code operates by creating an estimate of the true state called Xfilter (named after the Kalman Filter), which is the estimate of the true state. 
  
- If there isn't an observation available, then we make a forecast of Xfilter at the next step, e.g.

```
Xforecast = np.dot( Am, Xfilter[:,i-1] ) 
# part of the if statement we don't visit if no obs...
Xfilter[:,i] = Xforecast 
```

- If there is an observation to use then we update this using the observations we have 

``` 
Xforecast = np.dot( Am, Xfilter[:,i-1] ) 
Xfilter[:,i] = Xforecast*(1-K) + Zobs[:,i]*K
```

- This is why the estimate has a very "spiky" nature to it, we're updating suddenly at specific times which creates sharp jumps. 